<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Longformer_11_12000_32_768_weighted_loss_function_02_2e_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q scikit-learn
!pip install -q transformers datasets  # transformers and datasets are Hugging Face libraries
!pip install -q wandb

import json
import numpy as np
import os
import pandas as pd
import pickle
import sys
import time
import torch
import wandb

from datasets              import DatasetDict
from google.colab          import auth, drive, files, userdata
from huggingface_hub       import create_repo, login, upload_file, hf_hub_download
from huggingface_hub.utils import RepositoryNotFoundError
from sklearn.metrics       import accuracy_score, average_precision_score, classification_report, f1_score, precision_score, precision_recall_fscore_support, recall_score, roc_auc_score
from torch.utils.data      import DataLoader
from tqdm.auto             import tqdm
from transformers          import AdamW, EvalPrediction, LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments
from torch.nn              import BCEWithLogitsLoss


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os
from google.colab    import userdata
from huggingface_hub import login, hf_hub_download

# Hugging Face Authenticate
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
login(token=hf_token)

# Verify
!huggingface-cli whoami

file_path = hf_hub_download(
    repo_id   ="claudelepere/skill_classification",
    repo_type = "dataset",
    filename  = "test_model_eval_results.csv"
)
print(f"file_path: {file_path}")  # /root/.cache/huggingface/hub/datasets--claudelepere--skill_classification/snapshots/51ead81f69b1689fc19694b3f034585cde9f56e1/test_model_eval_results.csv

# Next, open in Colab or download to local

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


claudelepere


test_model_eval_results.csv:   0%|          | 0.00/15.4k [00:00<?, ?B/s]

file_path: /root/.cache/huggingface/hub/datasets--claudelepere--skill_classification/snapshots/2049195448fa56df6845838825b2841c94a452ef/test_model_eval_results.csv


In [3]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
!python -V

print(f"currentdir: {os.getcwd()}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

datasetDict_zip_file_name = "dataset_11_12000.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]
print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()

# OOM: reduce batch size
#      small sizes (1 to 32):            PROs: better generalization in some cases
#                                        CONs: may produce noisier gradients
#      large sizes (128, 256, or higer): PROs: gradients are smoother, leading to more stable training
#                                        CONs: poorer generalization (overfitting) in some cases
#      intermediate sizes (32, 64):      combines the benefits of small and large sizes
batch_size = 8

# OOM: enable gradient accumulation to compensate for smaller batch sizes by accumulating gradients over several steps
#      effective batch size = per-device batch size x gradient accumulation steps;
#      in each iteration, the model computes the gradients, these gradients are immediately used to update the model parameters
gradient_accumulation_steps = 4  #<<<<<<<<<<<<<<<<<<< gradient_accumulation_steps may not be None => comment it in TrainingArguments

# OOM: use PYTORCH_CUDA_ALLOC_CONF to handle memory fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# OOM: check for zombie processes
if torch.cuda.is_available():
  !nvidia-smi
  torch.cuda.memory_summary()
!ps aux | grep python
#!kill -9 <PID>
#!nvidia-smi     # Checked if killed

# OOM: use fp16 (half precision) mixed precision training
#      reduces memory requirements by up to 50%
fp16 = True

# OOM: limit the number of GPU workers: 0 (default) or 1 in Colab
#dataloader_num_workers = 1

# OOM: reduce model size or input tokens
#      1) LongformerTokenizer.from_pretrained('allenai/longformer-base/large-4096'): large/base: 435M/149M parameters
#      2) max_length: 4096 max for Longformer; 1 word can give several tokens, stop words are NOT discarded!
#         word_text_length_counts_sorted: jobs count                 : 50000
#                                         jobs count under  512 words: 44794  89.59%
#                                         jobs count under  640 words: 47894  95.79%
#                                         jobs count under  768 words: 49123  98.25%
#                                         jobs count under  896 words: 49691  99.38%
#                                         jobs count under 1024 words: 49917  99.83%
#                                         jobs count under 2048 words: 50000 100.00%
#                                         jobs count under 4096 words: 50000 100.00%
max_length = 768

# OOM: free up GPU memory
torch.cuda.empty_cache()

# OOM: monitor GPU memory usage
#!nividia-smi

# 1 epoch is a complete pass through the entire training dataset;
# with n datapoints and batch size = b, n/b iterations to complete 1 epoch;
# 1 iteration is a single update of the model's parameters
epochs = 5

# A common rule is to scale the learning rate proportionaly with the effective batch size
# note: get_linear_schedule_with_warmup <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
learning_rate = 2e-5  # 1e-5 x 32/8

# Reduce the number of transformers layers
#hidden_layers = 12    # 12 (default) or 6

# Threshold: 0.5 (default)
threshold = 0.2

if fp16:
  _fp = "fp16"
else:
  _fp = "fp32"

if 'gradient_accumulation_steps' not in globals():
  run_name = f"Longformer-multilabel-{datasetDict_dir_name}-length{max_length}-batch{batch_size}-epochs{epochs}-lr{learning_rate}-{_fp}-threshold{threshold}"
else:
  run_name = f"Longformer-multilabel-{datasetDict_dir_name}-length{max_length}-batch{batch_size}x{gradient_accumulation_steps}-epochs{epochs}-lr{learning_rate}-{_fp}-threshold{threshold}"
print(f"run_name                 : {run_name}")

Python 3.10.12
currentdir: /content
device: cuda
datasetDict_zip_file_name: dataset_11_12000.zip
datasetDict_dir_name     : dataset_11_12000

Sun Jan  5 11:49:38 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0              42W / 400W |      5MiB / 40960MiB |      0%      Default |
|               

In [4]:
def upload_unzip_dataset(file_name=datasetDict_zip_file_name):
  # Check if the file exists
  if not os.path.exists(file_name):
    print(f"'{file_name}' not found in /content. Uploading...")
    uploaded_files = files.upload()                              # Prompt file upload dialog
    if file_name not in uploaded_files:
      raise FileNotFoundError(f"'{file_name}' was not uploaded. Please try again.")
    print(f"'{file_name}' successfully uploaded to /content")
    uploaded_file_name = list(uploaded_files.keys())[0]          # Get the name of the uploaded file

    !unzip {uploaded_file_name}

    unzipped_dir_name = os.path.splitext(uploaded_file_name)[0]
    assert unzipped_dir_name==datasetDict_dir_name, "unzipped_dir_name != datasetDict_dir_name"
  else:
    print(f"'{datasetDict_dir_name}' already exists in /content.")


In [5]:
upload_unzip_dataset(datasetDict_zip_file_name)

'dataset_11_12000.zip' not found in /content. Uploading...


Saving dataset_11_12000.zip to dataset_11_12000.zip
'dataset_11_12000.zip' successfully uploaded to /content
Archive:  dataset_11_12000.zip
  inflating: dataset_11_12000/dataset_dict.json  
  inflating: dataset_11_12000/test/data-00000-of-00001.arrow  
  inflating: dataset_11_12000/test/dataset_info.json  
  inflating: dataset_11_12000/test/state.json  
  inflating: dataset_11_12000/train/data-00000-of-00001.arrow  
  inflating: dataset_11_12000/train/dataset_info.json  
  inflating: dataset_11_12000/train/state.json  
  inflating: dataset_11_12000/validation/data-00000-of-00001.arrow  
  inflating: dataset_11_12000/validation/dataset_info.json  
  inflating: dataset_11_12000/validation/state.json  


In [6]:
# Hugging Face Authenticate

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')
login(token=hf_token)

# Verify
!huggingface-cli whoami

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


claudelepere


In [7]:
# Create the skill_classification repo on the Hugging Face Hub

HF_name         = "claudelepere/skill_classification"
repo_id_model   = HF_name
repo_id_dataset = HF_name

repo_model_url = create_repo(
    repo_id   = repo_id_model,
    repo_type = "model",
    private   = True,
    exist_ok  = True
)
print(f"Repo model url: {repo_model_url} created successfully as a private repo.")

repo_dataset_url = create_repo(
    repo_id   = repo_id_dataset,
    repo_type = "dataset",
    private   = True,
    exist_ok  = True
)
print(f"Repo datasets url: {repo_dataset_url} created successfully as a private repo.")

repo_id_dataset = f"datasets/{HF_name}"

print(f"repo_id_model: {repo_id_model}")
print(f"repo_id_dataset: {repo_id_dataset}")

Repo model url: https://huggingface.co/claudelepere/skill_classification created successfully as a private repo.
Repo datasets url: https://huggingface.co/datasets/claudelepere/skill_classification created successfully as a private repo.
repo_id_model: claudelepere/skill_classification
repo_id_dataset: datasets/claudelepere/skill_classification


In [8]:
# W&B initialization

os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

try:
  wandb.init(
      project = "skill_classification",
      name    = run_name,
      entity  = "claudelepere-c-cile-cy",
      config  = {
          "learning_rate": learning_rate,
          "epochs"       : 5,
          "batch_size"   : batch_size
      }
  )
except wandb.errors.CommError as err:
  print(f"CommError: {err}")
except Exception as exc:
  print(f"Exception: {exc}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: claudelepere (claudelepere-c-cile-cy). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
# Create the dataset: 3 Hugging Face Dataset in a Hugging Face DatasetDict

datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)

print(f"datasetDict: {type(datasetDict)} {datasetDict.shape}\n{datasetDict}")
print(f"datasetDict.keys(): {datasetDict.keys()}")
print(f"datasetDict['train']: {type(datasetDict['train'])} {datasetDict['train'].shape}")
print(f"datasetDict['validation']: {type(datasetDict['validation'])} {datasetDict['validation'].shape}")
print(f"datasetDict['test']: {type(datasetDict['test'])} {datasetDict['test'].shape}")

datasetDict: <class 'datasets.dataset_dict.DatasetDict'> {'train': (9600, 8), 'validation': (1200, 8), 'test': (1200, 8)}
DatasetDict({
    train: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['id', 'text', '390', '135', '136', '137', '138', '139'],
        num_rows: 1200
    })
})
datasetDict.keys(): dict_keys(['train', 'validation', 'test'])
datasetDict['train']: <class 'datasets.arrow_dataset.Dataset'> (9600, 8)
datasetDict['validation']: <class 'datasets.arrow_dataset.Dataset'> (1200, 8)
datasetDict['test']: <class 'datasets.arrow_dataset.Dataset'> (1200, 8)


In [10]:
example = datasetDict['train'][0]
print(f"datasetDict['train'][0]: {type(example)} {example.keys()}\n{example}")

datasetDict['train'][0]: <class 'dict'> dict_keys(['id', 'text', '390', '135', '136', '137', '138', '139'])
{'id': 153918, 'text': "Proximus SpearIT - Senior Web Security Engineer F5, Security, Firewall, LTM, ASM Proximus SpearIT Job description Install, deploy, manage and operate security solutions based on Web Application Firewall systems Mandatory knowledge of F5 Technologies (LTM and ASM) Good Knowledge of Firewall, Proxy, Networking (Routing & Switching) systems and technologies Testing of new configuration, equipment and releases Reporting on network/security usage and performance Participate to project and deployment of new security architecture Deliver technical solutions and improve the level of protection of the system Develop the technical documentation regarding the operational procedures Be part in the security incident response process and participate to On-Duty staffing outside business hours. Job requirements Advanced skills in F5 (LTM and ASM) architectures, solutions 

In [11]:
# Create the label list and the id2label and label2id mappings.

"""
dataset 7_1000_125_125  ,  48 labels
dataset 7_128_18_54     ,  42 labels
dataset 8910_1087_68_204, 206 labels
dataset 11_1000         ,   6 labels
"""

labels = [label for label in datasetDict['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")

id2label = {idx:label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

label2id = {label:idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

labels: <class 'list'> 6
['135', '136', '137', '138', '139', '390']
id2label: <class 'dict'> 6
{0: '135', 1: '136', 2: '137', 3: '138', 4: '139', 5: '390'}
label2id: <class 'dict'> 6
{'135': 0, '136': 1, '137': 2, '138': 3, '139': 4, '390': 5}


In [12]:
# Load tokenizer and model

model_name = "allenai/longformer-base-4096"

tokenizer = LongformerTokenizerFast.from_pretrained(model_name)

model = LongformerForSequenceClassification.from_pretrained(
    model_name,
    num_labels        = len(labels),
#    num_hidden_layers = hidden_layers,
    problem_type      = 'multi_label_classification')

# Configure attention window size
model.config.attention_window = 512

optimizer = AdamW(model.parameters(), lr=learning_rate)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
# Tokenize ('input_ids' and 'attention_mask'), add 'global_attention_mask' (for Longformer), add 'labels'

def preprocess_data(examples, indices):
  # Step 1: Extract text and tokenize
  text = examples['text']             # Batch of texts
  encoding = tokenizer(
      text,                           # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = max_length,
      return_tensors = 'pt'           # Return PyTorch tensors
  )

  # Step 2: Create and add the global attention mask
  global_attention_mask             = torch.zeros_like(encoding['input_ids'])  # Initialize global attention mask with zeros (same shape as input_ids)
  global_attention_mask[:, 0]       = 1                                        # Set global attention on the first token ([CLS], token ID=0) in each sequence
  encoding['global_attention_mask'] = global_attention_mask                    # Add the global_attention_mask to the batch

  # Step 3: Create and populate the label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)   # Create an empty label matrix
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")
  #---------Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  encoding['labels'] = labels_matrix                                           # Add labels to the encoding
  #print(f"encoding['labels']: {encoding['labels']}")

  # encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
  #   'input_ids': tensor([[
  #   'attention_mask': tensor([[
  #   'global_attention_mask': tensor([[
  #   'labels': tensor([[
  print(f"encoding: {type(encoding)} {encoding.keys()}\n{encoding}")

  return encoding

In [14]:
# Create the 3 encoded datasets, train, validation and test

encoded_dataset = datasetDict.map(
    preprocess_data,
    batched        = True,
    remove_columns = datasetDict['train'].column_names,
    with_indices   = True
    )
train_dataset      = encoded_dataset['train']
validation_dataset = encoded_dataset['validation']
test_dataset       = encoded_dataset['test']

print(f"encoded_dataset: {type(encoded_dataset)} {encoded_dataset.shape}\n{encoded_dataset}")
print(f"train_dataset: {type(train_dataset)} {train_dataset.shape} {train_dataset.features}")
print(f"validation_dataset: {type(validation_dataset)} {validation_dataset.shape} {validation_dataset.features}")
print(f"test_dataset: {type(test_dataset)} {test_dataset.shape} {test_dataset.features}")
print("---")
print(f"test_dataset['labels']: {type(test_dataset['labels'])} {len(test_dataset['labels'])}\n{test_dataset['labels']}")
print("---")
print(f"train_dataset[0]['input_ids']: {type(train_dataset[0]['input_ids'])} {len(train_dataset[0]['input_ids'])}\n{train_dataset['input_ids'][0]}")
print(f"train_dataset[0]['attention_mask']: {type(train_dataset[0]['attention_mask'])} {len(train_dataset[0]['attention_mask'])}\n{train_dataset['attention_mask'][0]}")
print(f"train_dataset[0]['global_attention_mask']: {type(train_dataset[0]['global_attention_mask'])} {len(train_dataset[0]['global_attention_mask'])}\n{train_dataset['global_attention_mask'][0]}")
print(f"train_dataset[0]['labels']: {type(train_dataset[0]['labels'])} {len(train_dataset[0]['labels'])}\n{train_dataset[0]['labels']}")
print(f"train_dataset['labels'][0]: {type(train_dataset['labels'][0])} {len(train_dataset['labels'][0])}\n{train_dataset['labels'][0]}")


Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
{'input_ids': tensor([[    0, 10653,  1178,  ...,     1,     1,     1],
        [    0,  6209, 14286,  ...,     1,     1,     1],
        [    0, 43218,    29,  ...,     1,     1,     1],
        ...,
        [    0,  1000,  2253,  ...,     1,     1,     1],
        [    0,  1000,  2253,  ...,     1,     1,     1],
        [    0, 10653,  1178,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'global_attention_mask': tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0,

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
{'input_ids': tensor([[    0,  1000,  2253,  ...,     1,     1,     1],
        [    0,  8138, 15354,  ...,     1,     1,     1],
        [    0,   846,  6837,  ...,     1,     1,     1],
        ...,
        [    0,  1121, 38823,  ...,     1,     1,     1],
        [    0, 45395,   354,  ...,     1,     1,     1],
        [    0,   846,  6837,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'global_attention_mask': tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0,

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
{'input_ids': tensor([[    0,   846,  3006,  ...,     1,     1,     1],
        [    0,  9089,  1641,  ...,     1,     1,     1],
        [    0,   104, 26142,  ...,     1,     1,     1],
        ...,
        [    0, 45395,   354,  ...,     1,     1,     1],
        [    0, 36812, 16446,  ...,     1,     1,     1],
        [    0, 17085,  1000,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'global_attention_mask': tensor([[1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        ...,
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0,

In [15]:
# Truncated part

def get_truncated_part(text):
  tokens = tokenizer(
      text,
      truncation                = True,
      padding                   = 'max_length',
      max_length                = max_length,
      return_overflowing_tokens = True,
      return_tensors            = None
      )
  print(f"tokens.keys(): {tokens.keys()}")

  # Get the truncated tokens
  truncated_ids = tokens["input_ids"][0]
  print(f"truncated_ids: {type(truncated_ids)} {truncated_ids}")
  #overflow_ids  = tokens["overflow_to_sample_mapping"][0]
  #print(f"overflow_ids: {type(overflow_ids)} {overflow_ids}")

  # Decode the tokens back to text
  truncated_text = tokenizer.decode(truncated_ids, skip_special_tokens=True)
  #overflow_text  = tokenizer.decode(overflow_ids, skip_special_tokens=True)

  print(f"original_text :\n{text}")
  print(f"truncated_text:\n{truncated_text}")
  #print(f"overflow_text:\n{overflow_text}")

  original_tokens  = tokenizer.tokenize(text)
  truncated_tokens = tokenizer.tokenize(truncated_text)
  #overflow_tokens  = tokenizer.tokenize(overflow_text)

  print(f"original_tokens count : {len(original_tokens)}")
  print(f"truncated_tokens count: {len(truncated_tokens)}")
  #print(f"overflow_tokens count: {len(overflow_tokens)}")


In [16]:
example_text = datasetDict['train'][0]['text']
#get_truncated_part(example_text)

inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

# inputs: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask'])
#   {'input_ids': tensor([[
#   'attention_mask': tensor([[
#print(f"inputs: {type(inputs)} {inputs.keys()}") #\n{inputs}")
#print(f"inputs_ids: {type(inputs.input_ids)} {inputs.input_ids.shape}\n{inputs.input_ids}")
#print(f"attention_mask: {type(inputs.attention_mask)} {inputs.attention_mask.shape}\n{inputs.attention_mask}")
#print(f"labels: {inputs.labels.shape}")

In [17]:
# 4. Forward pass for multi-label classification

outputs = model(
    input_ids      = inputs.input_ids,
    attention_mask = inputs.attention_mask
    )

print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

# Logits (= raw model outputs)
logits = outputs.logits
print(f"logits: {type(logits)} {logits.shape}\n{logits}")

# Convert logits to probabilities
sigmoid = torch.nn.Sigmoid()
probs   = sigmoid(logits)
print(f"probs: {type(probs)} {probs.shape}\n{probs}")

Initializing global attention on CLS token...
Input ids are automatically padded to be a multiple of `config.attention_window`: 512


outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['logits'])
LongformerSequenceClassifierOutput(loss=None, logits=tensor([[ 0.0768,  0.0906, -0.0167, -0.1801,  0.0238, -0.1364]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)
logits: <class 'torch.Tensor'> torch.Size([1, 6])
tensor([[ 0.0768,  0.0906, -0.0167, -0.1801,  0.0238, -0.1364]],
       grad_fn=<AddmmBackward0>)
probs: <class 'torch.Tensor'> torch.Size([1, 6])
tensor([[0.5192, 0.5226, 0.4958, 0.4551, 0.5060, 0.4660]],
       grad_fn=<SigmoidBackward0>)


In [18]:
example = encoded_dataset['train'][0]

print(f"example: {type(example)} {example.keys()}\n{example}")
print()
print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

example: <class 'dict'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
{'input_ids': [0, 10653, 1178, 40439, 37828, 2068, 111, 3596, 6494, 2010, 24379, 274, 245, 6, 2010, 6, 1833, 12764, 6, 226, 14386, 6, 6015, 448, 1698, 1178, 40439, 37828, 2068, 13576, 8194, 34473, 6, 11386, 6, 3616, 8, 4303, 573, 2643, 716, 15, 6494, 11199, 1833, 12764, 1743, 28666, 2655, 9, 274, 245, 5974, 36, 574, 14386, 8, 6015, 448, 43, 2497, 27831, 9, 1833, 12764, 6, 45571, 6, 3658, 154, 36, 500, 25754, 359, 11171, 154, 43, 1743, 8, 4233, 25980, 9, 92, 20393, 6, 2104, 8, 8255, 22100, 15, 1546, 73, 15506, 9453, 8, 819, 31013, 877, 7, 695, 8, 9737, 9, 92, 573, 9437, 33092, 3165, 2643, 8, 1477, 5, 672, 9, 2591, 9, 5, 467, 15347, 5, 3165, 14877, 2624, 5, 5903, 6196, 1456, 233, 11, 5, 573, 1160, 1263, 609, 8, 4064, 7, 374, 12, 495, 27237, 15996, 751, 265, 722, 4, 13576, 3471, 12386, 2417, 11, 274, 245, 36, 574, 14386, 8, 6015, 448, 43, 41885, 6, 2643, 8, 785, 1698, 2987, 2038, 676, 9, 

In [19]:
tokenizer.decode(example['input_ids'])

"<s>Proximus SpearIT - Senior Web Security Engineer F5, Security, Firewall, LTM, ASM Proximus SpearIT Job description Install, deploy, manage and operate security solutions based on Web Application Firewall systems Mandatory knowledge of F5 Technologies (LTM and ASM) Good Knowledge of Firewall, Proxy, Networking (Routing & Switching) systems and technologies Testing of new configuration, equipment and releases Reporting on network/security usage and performance Participate to project and deployment of new security architecture Deliver technical solutions and improve the level of protection of the system Develop the technical documentation regarding the operational procedures Be part in the security incident response process and participate to On-Duty staffing outside business hours. Job requirements Advanced skills in F5 (LTM and ASM) architectures, solutions and products Proven professional experience of several years (>4 years) in a similar operational environment Master's degree in 

In [20]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['137', '138']

In [21]:
# Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines

# The 3 Hugging Face Dataset are formatted as PyTorch Dataset
encoded_dataset.set_format('torch')

In [22]:
batch_size  = batch_size
metric_name = "f1"

In [23]:
training_args = TrainingArguments(
    output_dir                  = './training_results',  # where model predictions and checkpoints will be written during training
    overwrite_output_dir        = True,
    logging_dir                 = './logs',
    logging_steps               = 50,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = 'epoch',
    save_strategy               = 'epoch',
    learning_rate               = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    num_train_epochs            = epochs,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    run_name                    = run_name,
    fp16                        = fp16,
    #dataloader_num_workers      = dataloader_num_workers,
    report_to                  = 'wandb'
    )

In [24]:
# Metrics
#   source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

def multi_label_metrics(predictions, labels):
    average = 'micro'    # 'micro' or 'weighted'

    # first, apply sigmoid on predictions whose shape is (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(torch.Tensor(predictions))

    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # finally, compute metrics
    y_true               = labels
    f1                   = f1_score               (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    precision            = precision_score        (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    recall               = recall_score           (y_true=y_true, y_pred=y_pred, average=average)    #, zero_division=1)
    roc_auc              = roc_auc_score          (y_true=y_true, y_score=probs, average=average)
    precision_recall_auc = average_precision_score(y_true=y_true, y_score=probs, average=average)
    accuracy             = accuracy_score         (y_true=y_true, y_pred=y_pred)

    # return as dictionary
    metrics = {
        'f1'                  : f1,
        'precision'           : precision,
        'recall'              : recall,
        'roc_auc'             : roc_auc,
        'precision_recall_auc': precision_recall_auc,
        'accuracy'            : accuracy
        }

    return metrics

In [25]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions = preds,
        labels      = p.label_ids
        )
    return result

Let's verify a batch as well as a forward pass:

In [26]:
print(f"input_ids:              {type(encoded_dataset['train']['input_ids'][0])}\t{encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask:         {type(encoded_dataset['train']['attention_mask'][0])}\t{encoded_dataset['train']['attention_mask'][0].shape}")
print(f"global_attention_mask:  {type(encoded_dataset['train']['global_attention_mask'][0])}\t{encoded_dataset['train']['global_attention_mask'][0].shape}")
print(f"labels:                 {type(encoded_dataset['train'][0]['labels'])}\t{encoded_dataset['train'][0]['labels'].shape}")

input_ids:              <class 'torch.Tensor'>	torch.Size([768])
attention_mask:         <class 'torch.Tensor'>	torch.Size([768])
global_attention_mask:  <class 'torch.Tensor'>	torch.Size([768])
labels:                 <class 'torch.Tensor'>	torch.Size([6])


In [27]:
# Execute a forward pass for debugging or verification purposes (cf. BERT_3_1 in Notion BERT database)

outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
    )

print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

outputs: <class 'transformers.models.longformer.modeling_longformer.LongformerSequenceClassifierOutput'> odict_keys(['loss', 'logits'])
LongformerSequenceClassifierOutput(loss=tensor(0.7155, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0768,  0.0906, -0.0167, -0.1801,  0.0238, -0.1364]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None, global_attentions=None)


In [28]:
# Define the weighted loss function

class_weights = torch.tensor([7.68, 2.15, 0.61, 0.47, 0.68, 6.26], dtype=torch.float32).to(device)
loss_fn       = BCEWithLogitsLoss(pos_weight=class_weights)  # For multi-label classification (binary classification per label)

In [29]:
# CustomTrainer

class CustomTrainer(Trainer):

  # No print in compute_loss because out of memory because prints are batch per batch
  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

    #print(f"inputs passed to compute_loss: {inputs.keys()}")
    input_ids             = inputs['input_ids']                        # shape: batch_size, sequence_length
    attention_mask        = inputs['attention_mask']                   # shape: batch_size, sequence_length
    global_attention_mask = inputs.get('global_attention_mask', None)  # shape: batch_size, sequence_length; optional as LongFormer specific
    labels                = inputs.get('labels', None)                 # shape: batch_size, num_labels; needed for loss computation, not required by the model

    #outputs = model(**inputs, global_attention_mask=global_attention_mask)  # Forward pass
    # Forward pass
    outputs = model(
        input_ids             = input_ids,
        attention_mask        = attention_mask,
        global_attention_mask = global_attention_mask,
        labels                = labels
    )
    #print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")
    logits = outputs.logits  # shape: (batch_size, num_labels)

    # If labels are provided, compute loss
    if labels is not None:
      loss = loss_fn(logits, labels)  # Compute weighted loss
      return (loss, outputs) if return_outputs else loss

    # If no labels, return outputs only, for evaluation or inference
    return outputs

In [30]:
# Create a Hugging Face's transformers trainer (which abstracts the training loop)

trainer = CustomTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = encoded_dataset["train"],
    eval_dataset    = encoded_dataset["validation"],
    compute_metrics = compute_metrics  # Optional: custom metrics function
    )


In [31]:
# Train

trainer_train = trainer.train()
print(f"trainer_train: {type(trainer_train)} len={len(trainer_train)}\n{trainer_train}")

file_path = "trainer_train.json"
with open(file_path, "w") as f:
  json.dump(trainer_train, f)
print(f"Train output successfully saved to {file_path}.")


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Roc Auc,Precision Recall Auc,Accuracy
1,2.048300,0.526245,0.776829,0.654080,0.956293,0.885142,0.821484,0.227500
2,1.700900,0.474277,0.796106,0.676315,0.967466,0.906634,0.846001,0.225833
3,1.445800,0.464135,0.816971,0.730261,0.927046,0.914830,0.846121,0.370000
4,1.436100,0.449257,0.816918,0.724568,0.936247,0.919701,0.853507,0.355833
5,1.219000,0.463206,0.828842,0.744375,0.934933,0.923157,0.857161,0.382500


trainer_train: <class 'transformers.trainer_utils.TrainOutput'> len=3
TrainOutput(global_step=1500, training_loss=1.6752266362508137, metrics={'train_runtime': 2414.3596, 'train_samples_per_second': 19.881, 'train_steps_per_second': 0.621, 'total_flos': 2.3647338381312e+16, 'train_loss': 1.6752266362508137, 'epoch': 5.0})
Train output successfully saved to trainer_train.json.


In [32]:
print("Training successfully completed.")

Training successfully completed.


## Upload model, tokenizer, train results, evaluate results

In [33]:
# Save model to /content

model_path = "model"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.json',
 'model/merges.txt',
 'model/added_tokens.json',
 'model/tokenizer.json')

In [34]:
# Upload model and tokenizer to HF repo_id_model

tokenizer = LongformerTokenizerFast.from_pretrained(model_path)
model     = LongformerForSequenceClassification.from_pretrained(model_path)

tokenizer.push_to_hub(repo_id_model)
model.push_to_hub(repo_id_model)


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/claudelepere/skill_classification/commit/40fb1c11a2024e72e7069f8015d370096dfe7e3a', commit_message='Upload LongformerForSequenceClassification', commit_description='', oid='40fb1c11a2024e72e7069f8015d370096dfe7e3a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='model', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

In [35]:
# Upload Train results to HF repo_id_dataset

# Train
upload_file(
    path_or_fileobj = 'trainer_train.json',
    path_in_repo    = 'trainer_train.json',
    repo_id         = HF_name,
    repo_type       = 'dataset'
)

CommitInfo(commit_url='https://huggingface.co/datasets/claudelepere/skill_classification/commit/0658b32ebaeebe0f59936933b3b626edc279928a', commit_message='Upload trainer_train.json with huggingface_hub', commit_description='', oid='0658b32ebaeebe0f59936933b3b626edc279928a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

To Get Results of Evaluation and Test

In [36]:
def get_results(model, dataset, batch_size, threshold, phase):
  # Clear GPU cache
  torch.cuda.empty_cache()

  # Set the model to evaluation mode to disable dropout and other training-specific behaviors
  model.eval()

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  dataLoader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

  all_preds       = []
  all_probs       = []
  all_true_labels = []

  for batch in tqdm(dataLoader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
      outputs = model(**batch)
    logits = outputs.logits

    # Convert logits to probabilities and probabilities to predictions
    sigmoid = torch.nn.Sigmoid()
    probs   = sigmoid(logits).cpu().numpy()    # Convert to Numpy
    preds   = (probs > threshold).astype(int)  # Convert to binary Numpy array

    # Accumulate probabilities, predictions and labels
    all_probs.append(probs)
    all_preds.append(preds)
    all_true_labels.append(batch['labels'].cpu().numpy())

  # Concatenate results from all batches
  all_probs       = np.concatenate(all_probs, axis=0)        # shape: [num_samples, num_labels]
  all_preds       = np.concatenate(all_preds, axis=0)        # shape: [num_samples, num_labels]
  all_true_labels = np.concatenate(all_true_labels, axis=0)  # shape: [num_samples, num_labels]

  print(f"all_probs:       {type(all_probs)} {all_probs.shape}")
  print(f"all_preds:       {type(all_preds)} {all_preds.shape}")
  print(f"all_true_labels: {type(all_true_labels)} {all_true_labels.shape}")
  results_df = pd.DataFrame({
      'predictions'  : [list(pred)  for pred  in all_preds],
      'probabilities': [list(prob)  for prob  in all_probs],
      'true_labels'  : [list(label) for label in all_true_labels]
  })
  results_file_path = f"{phase}_results.csv"
  results_df.to_csv(results_file_path, index=False)

  # Classification report for precision, recall, F1 score
  report = classification_report(
      y_true        = all_true_labels,
      y_pred        = all_preds,
      target_names  = labels,
      zero_division = 0,
      output_dict   = True
  )
  print(f"Classification Report:\n{report}")

  # ROC AUC for multi-label classification
  roc_auc = roc_auc_score(
      y_true  = all_true_labels,
      y_score = all_probs,
      average = 'micro'
  )
  print(f"ROC AUC: {roc_auc}")

  metrics = {
      'classification_report': report,
      'roc_auc'              : roc_auc
  }
  metrics_file_path = f"{phase}_metrics.json"
  with open(metrics_file_path, "w") as f:
    json.dump(metrics, f, indent=4)

  print(f"{phase} Results Saved to {results_file_path}")
  print(f"{phase} Metrics Saved to {metrics_file_path}")

## Evaluate

After training, we evaluate our model on the validation set.

In [37]:
# First evaluate results

phase_evaluate = 'evaluate_model_eval'

get_results(
    model      = model,
    dataset    = validation_dataset,
    batch_size = batch_size,
    threshold  = threshold,
    phase      = phase_evaluate
)

  0%|          | 0/150 [00:00<?, ?it/s]

all_probs:       <class 'numpy.ndarray'> (1200, 6)
all_preds:       <class 'numpy.ndarray'> (1200, 6)
all_true_labels: <class 'numpy.ndarray'> (1200, 6)
Classification Report:
{'135': {'precision': 0.4166666666666667, 'recall': 0.6818181818181818, 'f1-score': 0.5172413793103449, 'support': 66.0}, '136': {'precision': 0.4797979797979798, 'recall': 0.8050847457627118, 'f1-score': 0.6012658227848101, 'support': 236.0}, '137': {'precision': 0.8460674157303371, 'recall': 0.9007177033492823, 'f1-score': 0.8725376593279258, 'support': 836.0}, '138': {'precision': 0.9306157849089333, 'recall': 0.9907663896583564, 'f1-score': 0.9597495527728086, 'support': 1083.0}, '139': {'precision': 0.654643823264202, 'recall': 0.979757085020243, 'f1-score': 0.7848648648648648, 'support': 741.0}, '390': {'precision': 0.3493975903614458, 'recall': 0.7160493827160493, 'f1-score': 0.46963562753036436, 'support': 81.0}, 'micro avg': {'precision': 0.7443746729461015, 'recall': 0.9349326322707854, 'f1-score': 0.82

In [38]:
print("First evaluation successfully completed.")

First evaluation successfully completed.


In [39]:
# Second evaluate results

trainer_evaluate = trainer.evaluate()
print(f"trainer_evaluate: {type(trainer_evaluate)} len={len(trainer_evaluate)}\n{trainer_evaluate}")

file_path = "trainer_evaluate.json"
with open(file_path, "w") as f:
  json.dump(trainer_evaluate, f)
print(f"Evaluate output successfully saved to {file_path}.")

trainer_evaluate: <class 'dict'> len=11
{'eval_loss': 0.4632055163383484, 'eval_f1': 0.8288419519300801, 'eval_precision': 0.7443746729461015, 'eval_recall': 0.9349326322707854, 'eval_roc_auc': 0.9231573016733688, 'eval_precision_recall_auc': 0.8571608744223957, 'eval_accuracy': 0.3825, 'eval_runtime': 18.0054, 'eval_samples_per_second': 66.647, 'eval_steps_per_second': 8.331, 'epoch': 5.0}
Evaluate output successfully saved to trainer_evaluate.json.


In [40]:
print("Second evaluation successfully completed.")

Second evaluation successfully completed.


In [41]:
# Upload Evaluate Results to HF repo_id_dataset

upload_file(
    path_or_fileobj = f"{phase_evaluate}_results.csv",
    path_in_repo    = f"{phase_evaluate}_results.csv",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = f"{phase_evaluate}_metrics.json",
    path_in_repo    = f"{phase_evaluate}_metrics.json",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = 'trainer_evaluate.json',
    path_in_repo    = 'trainer_evaluate.json',
    repo_id         = HF_name,
    repo_type       = 'dataset'
)


CommitInfo(commit_url='https://huggingface.co/datasets/claudelepere/skill_classification/commit/ff323c042fef2be2762b279129302c8fe646cac4', commit_message='Upload trainer_evaluate.json with huggingface_hub', commit_description='', oid='ff323c042fef2be2762b279129302c8fe646cac4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

## Test

In [42]:
# First test results

phase_test = 'test_model_eval'

get_results(
    model      = model,
    dataset    = test_dataset,
    batch_size = batch_size,
    threshold  = threshold,
    phase      = phase_test
)

  0%|          | 0/150 [00:00<?, ?it/s]

all_probs:       <class 'numpy.ndarray'> (1200, 6)
all_preds:       <class 'numpy.ndarray'> (1200, 6)
all_true_labels: <class 'numpy.ndarray'> (1200, 6)
Classification Report:
{'135': {'precision': 0.5078125, 'recall': 0.8024691358024691, 'f1-score': 0.6220095693779905, 'support': 81.0}, '136': {'precision': 0.46867167919799496, 'recall': 0.8130434782608695, 'f1-score': 0.5945945945945946, 'support': 230.0}, '137': {'precision': 0.8360836083608361, 'recall': 0.9325153374233128, 'f1-score': 0.8816705336426914, 'support': 815.0}, '138': {'precision': 0.9276257722859664, 'recall': 0.9859287054409006, 'f1-score': 0.9558890404729422, 'support': 1066.0}, '139': {'precision': 0.6789667896678967, 'recall': 0.983957219251337, 'f1-score': 0.8034934497816594, 'support': 748.0}, '390': {'precision': 0.46551724137931033, 'recall': 0.7641509433962265, 'f1-score': 0.5785714285714286, 'support': 106.0}, 'micro avg': {'precision': 0.7525476874836686, 'recall': 0.9455022980958634, 'f1-score': 0.83806198

In [43]:
print("First test successfully completed.")

First test successfully completed.


In [44]:
# Second test results

trainer_predict = trainer.predict(test_dataset)
print(f"trainer_predict: {type(trainer_predict)} len={len(trainer_predict)}\n{trainer_predict}")
print(f"trainer_predict.predictions: {type(trainer_predict.predictions)} shape={trainer_predict.predictions.shape}")  # Model logits
print(f"trainer_predict.label_ids: {type(trainer_predict.label_ids)} shape={trainer_predict.label_ids.shape}")        # Ground truth labels
print(f"trainer_predict.metrics: {type(trainer_predict.metrics)} len={len(trainer_predict.metrics)}")

trainer_predict_json_serializable = {
    'predictions': trainer_predict.predictions.tolist(),  # Convert Numpy array to list
    'label_ids'  : trainer_predict.label_ids.tolist(),    # Convert Numpy array to list
    'metrics'    : trainer_predict.metrics                # Dictionary is already serializable
}

file_path = "trainer_predict.json"
with open(file_path, "w") as f:
  json.dump(trainer_predict_json_serializable, f)
print(f"Test output successfully saved to {file_path}.")

trainer_predict: <class 'transformers.trainer_utils.PredictionOutput'> len=3
PredictionOutput(predictions=array([[-3.1699219 , -0.32885742,  2.0957031 ,  2.5       , -0.56640625,
        -2.2011719 ],
       [ 3.8671875 ,  3.0664062 , -0.01025391, -2.3183594 , -2.9472656 ,
         3.84375   ],
       [-3.6796875 , -0.75927734,  2.1503906 ,  2.7421875 ,  0.02281189,
        -2.7246094 ],
       ...,
       [-3.8613281 , -3.5683594 , -2.2988281 ,  1.7060547 ,  2.5253906 ,
        -3.7890625 ],
       [-3.8183594 , -3.59375   , -2.3613281 ,  1.7158203 ,  2.5078125 ,
        -3.7714844 ],
       [-4.3164062 , -2.2792969 ,  1.0800781 ,  3.0273438 ,  0.38916016,
        -3.6464844 ]], dtype=float32), label_ids=array([[0., 1., 1., 1., 0., 0.],
       [1., 1., 1., 0., 0., 1.],
       [0., 1., 1., 1., 1., 0.],
       ...,
       [0., 0., 1., 1., 1., 0.],
       [0., 0., 0., 1., 1., 0.],
       [0., 0., 1., 1., 1., 0.]], dtype=float32), metrics={'test_loss': 0.426077276468277, 'test_f1': 0.8380

In [45]:
print("Second test successfully completed.")

Second test successfully completed.


In [46]:
# Upload Test Results to HF repo_id_dataset

upload_file(
    path_or_fileobj = f"{phase_test}_results.csv",
    path_in_repo    = f"{phase_test}_results.csv",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = f"{phase_test}_metrics.json",
    path_in_repo    = f"{phase_test}_metrics.json",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = 'trainer_predict.json',
    path_in_repo    = 'trainer_predict.json',
    repo_id         = HF_name,
    repo_type       = 'dataset'
)

CommitInfo(commit_url='https://huggingface.co/datasets/claudelepere/skill_classification/commit/24969fd259f3a709517e4c99c6313bc9d81b49ba', commit_message='Upload trainer_predict.json with huggingface_hub', commit_description='', oid='24969fd259f3a709517e4c99c6313bc9d81b49ba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/claudelepere/skill_classification', endpoint='https://huggingface.co', repo_type='dataset', repo_id='claudelepere/skill_classification'), pr_revision=None, pr_num=None)

In [47]:
print("It's the end")

It's the end
